In [1]:

import os
import numpy as np
import torch
from tqdm import tqdm
import json
from dual_network import Dual3DCNN3, Dual3DCNN4, Dual3DCNN5, FlexibleDual3DCNN
import torch
from torch.utils.data import Dataset
import SimpleITK as sitk
import numpy as np
import glob
from utilities import create_list_from_master_json, read_json_file, split_data
import re
import glob
import random
from torch.utils.data import Dataset, DataLoader
import numpy as np
import SimpleITK as sitk
import torch
from utilities import list_patient_folders, prepare_data_nrrd, split_data
from monai.transforms import Compose, LoadImaged, EnsureChannelFirstd, Spacingd, ScaleIntensityd, SpatialPadd, CenterSpatialCropd, ScaleIntensityRanged
from monai.data import CacheDataset, DataLoader, Dataset
from monai.transforms import LoadImaged
from monai.data.image_reader import ITKReader
from monai.data import SmartCacheDataset
import random
import optuna
from optuna.integration import PyTorchLightningPruningCallback

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
# data_path_NEW = '/data/shahpouriz/DBP_newDATA/nrrd/oneCTperPatients/proton'
data_path = '/data/shahpouriz/DBP_DATA_total/nrrd/oneCTperPatinet/proton'

patient_list_NEW = list_patient_folders(data_path)
# Shuffle patient list if you want randomness
random.shuffle(patient_list_NEW)

# Define split sizes
total_patients = len(patient_list_NEW)
train_size = int(total_patients * 0.70)
val_size = int(total_patients * 0.20)
# The rest will be for the test set

# Split the patient list
train_patients = patient_list_NEW[:train_size]
val_patients = patient_list_NEW[train_size:train_size + val_size]
test_patients = patient_list_NEW[train_size + val_size:]

train_pct, train_rct, train_pos = prepare_data_nrrd(data_path, train_patients)
val_pct, val_rct, val_pos = prepare_data_nrrd(data_path, val_patients)
test_pct, test_rct, test_pos = prepare_data_nrrd(data_path, test_patients)

# Create dictionaries for each dataset
train_data = [{"plan": img, "repeat": tar, "pos": pos} for img, tar, pos in zip(train_pct, train_rct, train_pos)]
val_data = [{"plan": img, "repeat": tar, "pos": pos} for img, tar, pos in zip(val_pct, val_rct, val_pos)]
test_data = [{"plan": img, "repeat": tar, "pos": pos} for img, tar, pos in zip(test_pct, test_rct, test_pos)]


# Check the lengths of the sets
print("Number of training samples:", len(train_data))
print("Number of validation samples:", len(val_data))
print("Number of test samples:", len(test_data))
print(len(test_data)+len(val_data)+len(train_data))

Number of training samples: 65
Number of validation samples: 17
Number of test samples: 7
89


In [3]:

dim = 128
size = (dim, dim, dim)
pixdim = (3.0, 3.0, 3.0)
transforms = Compose([
        LoadImaged(keys=["plan", "repeat"], reader=ITKReader()),
        EnsureChannelFirstd(keys=["plan", "repeat"]),
        # ScaleIntensityd(keys=["plan", "repeat"]),
        Spacingd(keys=["plan", "repeat"], pixdim=pixdim, mode='trilinear'),
        SpatialPadd(keys=["plan", "repeat"], spatial_size=size, mode='constant'),  # Ensure minimum size
        CenterSpatialCropd(keys=["plan", "repeat"], roi_size=size),  # Ensure uniform size
    ])


train_ds = CacheDataset(data=train_data, transform=transforms, cache_rate=0.8, num_workers=1)
train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, num_workers=1)

val_ds = CacheDataset(data=val_data, transform=transforms, cache_rate=0.8, num_workers=1)
val_loader = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=1)


Loading dataset: 100%|██████████| 13/13 [00:00<00:00, 17.76it/s]


In [7]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import optuna
from optuna.trial import TrialState
import functools
from torch.optim.lr_scheduler import ReduceLROnPlateau


lambda_reg = 0.0  # for L1 regularization
weight_decay = 0.0  # for L2 regularization


def objective(trial, device, final_epoch):
    # Predefined layer configurations
    architectures = [
        [8, 16, 32],   # Small network
        [16, 32, 64],  # Small network

        [32, 64, 128],  # Medium network
        [16, 32, 64, 128],

        [32, 64, 128, 256],  # Larger network
        [16, 32, 128, 256, 512]  # Even larger network
    ]
    chosen_architecture = trial.suggest_categorical('architecture', architectures)
    
    model = FlexibleDual3DCNN(chosen_architecture).to(device)
    
    # Hyperparameters to optimize
    lr = trial.suggest_categorical('lr', [1e-5, 1e-4, 1e-3, 1e-2, 1e-1])
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam']) #, 'RMSprop', 'SGD'])
    loss_function = trial.suggest_categorical('loss_function', ['MSELoss', 'L1Loss']) #, 'SmoothL1Loss'])

    # Select optimizer
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=4, verbose=True)



    # Select loss function
    if loss_function == 'MSELoss':
        criterion = nn.MSELoss()
    elif loss_function == 'L1Loss':
        criterion = nn.L1Loss()
    elif loss_function == 'SmoothL1Loss':
        criterion = nn.SmoothL1Loss()



    # Training loop
    model.train()

    for epoch in range(final_epoch):
        model.train()
        loss_list = []
        for batch_data in train_loader:
            pCT, rCT = batch_data["plan"].to(device), batch_data["repeat"].to(device)
            reg = batch_data["pos"].clone().detach().requires_grad_(True).to(device)  # If gradients are required for 'reg'
            optimizer.zero_grad()

            output = model(pCT, rCT)
            loss = criterion(output, reg)


            # L1 Regularization
            l1_reg = torch.tensor(0., requires_grad=True).to(device)
            for name, param in model.named_parameters():
                # print(name, param)
                if 'weight' in name:  # Apply L1 only to weights, not biases
                    l1_reg = l1_reg + torch.norm(param, 1)
            
            # Combine loss with L1 regularization
            loss = loss + lambda_reg * l1_reg


            loss.backward()
            optimizer.step()

            loss_list.append(loss.item())
            mean_tot_loss = np.mean(loss_list)
            # print(f'Epoch: {epoch}/{final_epoch}, Batch: {i+1}/{len(train_loader)}, Loss_avg: {mean_tot_loss}')


        # Validation loop
        model.eval()
        val_loss = []
        with torch.no_grad():
            for batch_data in val_loader:
                pCT_val, rCT_val = batch_data["plan"].to(device), batch_data["repeat"].to(device)
                reg_val = batch_data["pos"].clone().detach().requires_grad_(True).to(device)  # If gradients are required for 'reg'

                output_val = model(pCT_val, rCT_val)
                loss_output_val = criterion(output_val, reg_val)

                val_loss.append(loss_output_val.item())

            mean_val_loss = np.mean(val_loss)
            # print(f'Epoch [{epoch+1}/{final_epoch}], Validation Loss: {mean_val_loss:.4f}')

            # Adjust learning rate
            scheduler.step(mean_val_loss)
    return mean_val_loss


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

final_epoch = 20 # The number of epochs



study = optuna.create_study(direction='minimize', study_name='DBP_optimize', storage='sqlite:///DBP_optimizer.db', load_if_exists=True)
study.optimize(lambda trial: objective(trial, device, final_epoch), n_trials=140) # the number of trials as needed

print("Best trial:")
print("  Loss: ", study.best_trial.value)
print("  Params: ")
for key, value in study.best_trial.params.items():
    print(f"    {key}: {value}")


[I 2024-03-18 12:09:55,204] A new study created in RDB with name: DBP_optimize
Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [8, 16, 32] which is of type list.
Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [16, 32, 64, 128] which is of type list.


Epoch 00013: reducing learning rate of group 0 to 1.0000e-06.
Epoch 00019: reducing learning rate of group 0 to 1.0000e-07.


[I 2024-03-18 12:13:46,757] Trial 0 finished with value: 0.631725579390631 and parameters: {'architecture': [32, 64, 128], 'lr': 1e-05, 'optimizer': 'Adam', 'loss_function': 'MSELoss'}. Best is trial 0 with value: 0.631725579390631.


Epoch 00006: reducing learning rate of group 0 to 1.0000e-05.
Epoch 00011: reducing learning rate of group 0 to 1.0000e-06.
Epoch 00016: reducing learning rate of group 0 to 1.0000e-07.


[I 2024-03-18 12:15:45,485] Trial 1 finished with value: 0.6514400508090415 and parameters: {'architecture': [16, 32, 64], 'lr': 0.0001, 'optimizer': 'Adam', 'loss_function': 'MSELoss'}. Best is trial 0 with value: 0.631725579390631.


...

Epoch 00013: reducing learning rate of group 0 to 1.0000e-06.
Epoch 00018: reducing learning rate of group 0 to 1.0000e-07.


[I 2024-03-18 19:51:07,700] Trial 138 finished with value: 0.6673847187091323 and parameters: {'architecture': [16, 32, 128, 256, 512], 'lr': 1e-05, 'optimizer': 'Adam', 'loss_function': 'L1Loss'}. Best is trial 105 with value: 0.6029359084718368.


Epoch 00009: reducing learning rate of group 0 to 1.0000e-06.
Epoch 00014: reducing learning rate of group 0 to 1.0000e-07.
Epoch 00019: reducing learning rate of group 0 to 1.0000e-08.


[I 2024-03-18 19:54:10,793] Trial 139 finished with value: 0.6434160946484875 and parameters: {'architecture': [16, 32, 128, 256, 512], 'lr': 1e-05, 'optimizer': 'Adam', 'loss_function': 'L1Loss'}. Best is trial 105 with value: 0.6029359084718368.


Best trial:
  Loss:  0.6029359084718368
  Params: 
    architecture: [16, 32, 64]
    lr: 1e-05
    optimizer: Adam
    loss_function: L1Loss


In [ ]:
# import functools

# def objective(trial, model, train_loader, val_loader, device, final_epoch, mae_loss):
#     lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    
#     optimizer = torch.optim.Adam(model.parameters(), lr=lr)
#     criterion = torch.nn.MSELoss()

#     # Training loop
#     model.train()

#     for epoch in range(starting_epoch, final_epoch):
#         model.train()  # Set model to training mode
#         mae_list = []
#         train_loss = []
#         for i, batch_data in enumerate(train_loader):  # Use enumerate to get the batch index
#             pCT, rCT = batch_data["plan"].to(device), batch_data["repeat"].to(device)
#             reg = batch_data["pos"].clone().detach().requires_grad_(True).to(device)  # If gradients are required for 'reg'
#             optimizer.zero_grad()

#             output = model(pCT, rCT)
#             loss_output = mae_loss(output, reg)

#             loss_output.backward()
#             optimizer.step()
            
#             # Logging
#             mae_list.append(loss_output.item())
#             mean_mae = np.mean(mae_list)
#             # Corrected to print the current batch number


#         # Validation loop
#         model.eval()
#         val_loss = []
#         with torch.no_grad():
#             for batch_data in val_loader:
#                 pCT_val, rCT_val = batch_data["plan"].to(device), batch_data["repeat"].to(device)
#                 reg_val = batch_data["pos"].clone().detach().requires_grad_(True).to(device)  # If gradients are required for 'reg'

#                 output_val = model(pCT_val, rCT_val)
#                 loss_output_val = mae_loss(output_val, reg_val)

#                 val_loss.append(loss_output_val.item())

#             mean_val_loss = np.mean(val_loss)
#     return mean_val_loss

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Use functools.partial to pass extra arguments to the objective function
# objective_with_args = functools.partial(
#     objective,
#     model=model,
#     train_loader=train_loader,
#     val_loader=val_loader,
#     device=device,
#     final_epoch=final_epoch,
#     mae_loss=mae_loss,

# )

# study = optuna.create_study(direction='minimize')
# study.optimize(objective_with_args, n_trials=5) 

# print("Best trial:", study.best_trial.number)
# print("Best value (validation loss):", study.best_value)
# print("Best hyperparameters:", study.best_params)

[I 2024-03-13 12:57:48,637] A new study created in memory with name: no-name-2cd05e81-8e1c-44d3-81a4-4be74941161f
[I 2024-03-13 12:58:47,573] Trial 0 finished with value: 0.309533763534079 and parameters: {'lr': 9.218304386771182e-06}. Best is trial 0 with value: 0.309533763534079.
[I 2024-03-13 12:59:54,444] Trial 1 finished with value: 0.3092626925185323 and parameters: {'lr': 6.610746908388528e-05}. Best is trial 1 with value: 0.3092626925185323.
[I 2024-03-13 13:01:01,542] Trial 2 finished with value: 0.30921216851100325 and parameters: {'lr': 2.4698381067064442e-05}. Best is trial 2 with value: 0.30921216851100325.
[I 2024-03-13 13:02:08,653] Trial 3 finished with value: 0.3096086760734518 and parameters: {'lr': 0.0007246235392743511}. Best is trial 2 with value: 0.30921216851100325.
[I 2024-03-13 13:03:15,620] Trial 4 finished with value: 0.3095567206541697 and parameters: {'lr': 1.2856561858081072e-05}. Best is trial 2 with value: 0.30921216851100325.
[I 2024-03-13 13:04:22,598] Trial 5 finished with value: 0.3095563782254855 and parameters: {'lr': 8.659948648935581e-08}. Best is trial 2 with value: 0.30921216851100325.
[I 2024-03-13 13:05:29,773] Trial 6 finished with value: 0.3095362058530251 and parameters: {'lr': 4.637608822923589e-06}. Best is trial 2 with value: 0.30921216851100325.
[I 2024-03-13 13:06:36,824] Trial 7 finished with value: 0.3088546040095389 and parameters: {'lr': 0.0002332795188029022}. Best is trial 7 with value: 0.3088546040095389.
[I 2024-03-13 13:07:44,302] Trial 8 finished with value: 0.3088908504260083 and parameters: {'lr': 0.0002125349283948824}. Best is trial 7 with value: 0.3088546040095389.
[I 2024-03-13 13:08:51,713] Trial 9 finished with value: 0.30929054742679 and parameters: {'lr': 0.0006778213291587758}. Best is trial 7 with value: 0.3088546040095389.
Best trial: 7
Best value (validation loss): 0.3088546040095389
Best hyperparameters: {'lr': 0.0002332795188029022}